In [1]:
import pandas as pd 

In [2]:
planillas_hist=pd.read_csv('0.0.planillas.hist.COMPLETE',index_col=0)

In [3]:
planillas_hist.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 265738 entries, 0 to 11694
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Dominio      265738 non-null  object
 1   Planilla     265738 non-null  int64 
 2   Taller       265738 non-null  int64 
 3   Tipo Uso     265738 non-null  object
 4   Fecha        265738 non-null  object
 5   Resultado    265738 non-null  object
 6   Versión RTO  265738 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 16.2+ MB


# Importar librerias

In [4]:
import time 

In [5]:
import requests

In [6]:
from tqdm.notebook import tqdm

In [7]:
from bs4 import BeautifulSoup

# Definir funciones

Request detalles de revision

In [8]:
def get_detalles(nro,codTaller,versionRTO):#input is in string format 
    global soup
    url = "https://rto.cent.gov.ar/rto/RTO/verDetallePlanillaRevision"

    payload = "nroPlanilla="+nro+"&codTaller="+codTaller+"&versionRTO="+versionRTO
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:108.0) Gecko/20100101 Firefox/108.0",
        "Accept": "text/javascript, text/html, application/xml, text/xml, */*",
        "Accept-Language": "es-AR,es;q=0.8,en-US;q=0.5,en;q=0.3",
        "Accept-Encoding": "gzip, deflate, br",
        "X-Requested-With": "XMLHttpRequest",
        "X-Prototype-Version": "1.6.1",
        "Content-type": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://rto.cent.gov.ar",
        "Connection": "keep-alive",
        "Referer": "https://rto.cent.gov.ar/rto/RTO/planillaDiaria",
        "Cookie": cookie,
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin"
    }

    response = requests.request("POST", url, data=payload, headers=headers)
    #print response text (test)
    #print(response.text)
    soup=BeautifulSoup(response.text,'lxml')

Limpiar soup para detalles 

In [9]:
def clean_data(tag):# returns a list of values: tags
        tags_datos=soup.select(tag) #data is inside the tag td
        global datos
        datos=[] 
        for data in tags_datos:
            datos.append(data.get_text(strip=True)) #make a list with the values 
        return(datos)

Limpiar soup para anomalias 

In [10]:
def clean_data_anomalias(tag_anomalias):# returns a list of values: tags
        tags_datos_anomalias=soup.select(tag_anomalias) #data is inside the tag td
        global datos_anomalias
        datos_anomalias=[] 
        for data in tags_datos_anomalias:
            datos_anomalias.append(data.get_text(strip=True)) #make a list with the values 
        return(datos_anomalias)

Crear lista de diccionarios con los detalles: cada diccionario es una fila 

In [11]:
def create_rows_detalles():
    global df
    global rows
    #make dictionary with only the relevant elements 
    #Search Label in the list,if it doesnt exist, returns empty string 
    if 'Número:'in datos:
        Planilla=datos[datos.index('Número:')+1]
    else:
        Planilla=''
    if 'Dominio:'in datos:
        Dominio=datos[datos.index('Dominio:')+1]
    else:
        Dominio=''
    if 'Año Modelo:'in datos:
        Modelo=datos[datos.index('Año Modelo:')+1]
    else:
        Modelo=''
    if 'Convenio:'in datos:
        Convenio=datos[datos.index('Convenio:')+1]
    else:
        Convenio=''
    if 'Resultado:'in datos:
        Resultado=datos[datos.index('Resultado:')+1]
    else:
        Resultado=''


    rows.append({'Planilla':Planilla,
     'Dominio:':Dominio,
     'Modelo':Modelo,
     'Convenio:':Convenio,
     'Resultado':Resultado
    })#make dict 

Crear lista de anomalias:Para cada anomalia es una fila

In [12]:
def create_rows_anomalias(nro_planilla):
    global df_anomalias
    global rows_anomalias
    if len(datos_anomalias)>0:
        a=int(0)
        for i in range (0,int((len(datos_anomalias))/4)):
            rows_anomalias.append([nro_planilla,datos_anomalias[a],datos_anomalias[a+1],datos_anomalias[a+2],datos_anomalias[a+3]])
            a+=int(4)
    else:
        rows_anomalias.append([nro_planilla,None,None,None,None])

Crear dataframe (df) de detalles 

In [13]:
def create_df_detalles():# returns a dataframe df 
    global df 
    df=pd.DataFrame(rows)
    return(df)

Crear dataframe (df_anomalias) de anomalias

In [14]:
def create_df_anomalias():# returns a dataframe df 
    global columns_anomalias
    global df_anomalias 
    columns_anomalias=['Planilla','Código','Gravedad','Descripción','Observaciones']
    df_anomalias=pd.DataFrame()
    df_anomalias=pd.DataFrame(rows_anomalias,columns=columns_anomalias)
    return(df_anomalias)

Utilizando las funciones anteriores. las unimos en una unica funcion. Iteramos en la lista de Nroplanillas y nroTaller

#  FUNCION SCRAPE

In [15]:
def scrape(lista_planillas):
    global rows
    global rows_anomalias
    rows=[]
    rows_anomalias=[] 
    tag='label'#tag detalles planilla  
    tag_anomalias='td'#tag anomalias
    for i in tqdm(lista_planillas):
        get_detalles(str(i[0]),str(i[1]),str(i[2]))
        clean_data(tag)
        create_rows_detalles()
        clean_data_anomalias(tag_anomalias)
        create_rows_anomalias(i[0])
    create_df_detalles()# returns dataframe df
    create_df_anomalias()#returns dataframe df_anomalias de anomalias 
    return([df['Planilla'].nunique(),df_anomalias['Planilla'].nunique()])

In [16]:
def scrape_test(lista_planillas):
    global rows
    global rows_anomalias
    rows=[]
    rows_anomalias=[] 
    tag='label'#tag detalles planilla  
    tag_anomalias='td'#tag anomalias
    for nro_planilla in tqdm(lista_planillas):
        get_detalles(nro_planilla[0],str(nro_planilla[1]))
        clean_data(tag)
        create_rows_detalles()
        clean_data_anomalias(tag_anomalias)
        create_rows_anomalias(nro_planilla[0])
    create_df_detalles()# returns dataframe df
    create_df_anomalias()#returns dataframe df_anomalias de anomalias 
    return([df['Planilla'].nunique(),df_anomalias['Planilla'].nunique()])

Guardar los dataframes en archivos .csv

In [17]:
def to_csv(num):
    filename=str(num)+'planillas.detalle'
    df.to_csv(filename)
    filename2=str(num)+'Anomalias'
    df_anomalias.to_csv(filename2)

Dividimos la lista de planillas en N listas. En caso de error no perdemos todo 

In [18]:
def div_lista(lista,part):
    global lista_part
    n=int(len(lista)/part)
    lista_part=[lista[i:i+n]for i in range(0,len(lista),n)]
    return(lista_part)
    
    

Definimos una nueva funcion Scrape que crea un csv por cada particion de la lista de planillas. Esto nos ayuda a no tener que comenzar de 0 si el server se cae o se corta internet 

In [19]:
def scrape_part(lista,part,start):
    div_lista(lista,part)
    for i in tqdm(range(0+start,part)):
        scrape(lista_part[i])
        to_csv(i)


Definimos una funcion para unir los archivos csv en un unico Dataframe y guardar como csv

In [20]:
def join_csv(part):
    global df_det
    global detalles
    global df_anom
    global anomalias
    detalles=pd.DataFrame()
    anomalias=pd.DataFrame()
    for i in range(0,part):
        df_det=pd.read_csv(str(i)+'planillas.detalle',index_col=0)
        detalles=pd.concat([detalles,df_det])
        df_anom=pd.read_csv(str(i)+'Anomalias',index_col=0)
        anomalias=pd.concat([anomalias,df_anom])        
    detalles.to_csv('0.0.planillas.detalle.COMPLETE')
    anomalias.to_csv('0.0.Anomalias.COMPLETE')
    


# Actualizar cookie

In [21]:
cookie="JSESSIONID=6B255F63B411B029C38BC4B028E1ECC4"

# Lista de planillas para test

In [22]:
planillas_lista=planillas_hist.loc[:,['Planilla','Taller','Versión RTO']]

In [23]:
planillas_lista=planillas_lista.values.tolist()

In [24]:
planillas_lista=planillas_lista[:5]
planillas_lista

[[1173505, 4, 2014],
 [1124182, 4, 2014],
 [1053004, 65, 2014],
 [1154432, 4, 2014],
 [1066213, 65, 2014]]

In [25]:
div_lista(planillas_lista,2)

[[[1173505, 4, 2014], [1124182, 4, 2014]],
 [[1053004, 65, 2014], [1154432, 4, 2014]],
 [[1066213, 65, 2014]]]

In [26]:
scrape_part(planillas_lista,2,0)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
df

,Planilla,Dominio:,Modelo,Convenio:,Resultado
0,1053004,AA000HG,2016,UP Nacional(Nacional-Argentina),Apto
1,1154432,AA001GK,2016,UP Chaco(Provincial-Chaco),Apto


In [28]:
df_anomalias

,Planilla,Código,Gravedad,Descripción,Observaciones
0,1053004,09010302,L,Carrocería > Exterior > Chapa Patente > Faltan...,ESTADO
1,1154432,09010302,L,Carrocería > Exterior > Chapa Patente > Patent...,CHAPA DELANTERA


In [29]:
join_csv(2)

In [30]:
detalles

,Planilla,Dominio:,Modelo,Convenio:,Resultado
0,1173505,AA000HG,2016,UP Chaco(Provincial-Chaco),Apto
1,1124182,AA000HG,2016,UP Chaco(Provincial-Chaco),Apto
0,1053004,AA000HG,2016,UP Nacional(Nacional-Argentina),Apto
1,1154432,AA001GK,2016,UP Chaco(Provincial-Chaco),Apto


In [31]:
anomalias

,Planilla,Código,Gravedad,Descripción,Observaciones
0,1173505,20502,L,"Tren Delantero, Tren Trasero, Suspensión > Amo...",AMORTIGUADORES TRASEROS
1,1173505,20506,L,"Tren Delantero, Tren Trasero, Suspensión > Amo...",AMORTIGUADORES TRASEROS
2,1173505,9010302,L,Carrocería > Exterior > Chapa Patente > Patent...,Patente en mal estado TRASERA
3,1124182,9010302,L,Carrocería > Exterior > Chapa Patente > Patent...,TRASERA
0,1053004,9010302,L,Carrocería > Exterior > Chapa Patente > Faltan...,ESTADO
1,1154432,9010302,L,Carrocería > Exterior > Chapa Patente > Patent...,CHAPA DELANTERA


# Scrape completo

In [32]:
planillas_lista=planillas_hist.loc[:,['Planilla','Taller','Versión RTO']] #numero de planillas: 265738
planillas_lista.drop_duplicates(inplace=True)#numero de planillas 265724
planillas_lista.value_counts()

Planilla  Taller  Versión RTO
14        249     2003           1
1137639   123     2014           1
1137618   124     2014           1
1137619   4       2014           1
1137620   4       2014           1
                                ..
1065672   50      2014           1
1065673   4       2014           1
1065674   4       2014           1
1065675   4       2014           1
1322406   129     2014           1
Length: 265724, dtype: int64

In [33]:
planillas_lista=planillas_lista.values.tolist()

In [35]:
len(planillas_lista)

265724

In [36]:
scrape_part(planillas_lista,100,96)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2657 [00:00<?, ?it/s]

  0%|          | 0/2657 [00:00<?, ?it/s]

  0%|          | 0/2657 [00:00<?, ?it/s]

  0%|          | 0/2657 [00:00<?, ?it/s]

In [41]:
join_csv(99)

# Leer csv completo y analizar

detalles

In [21]:
detalles=pd.read_csv('0.0.planillas.detalle.COMPLETE',index_col=0)
detalles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 257733 entries, 0 to 2656
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Planilla   236324 non-null  float64
 1   Dominio:   236324 non-null  object 
 2   Modelo     236324 non-null  float64
 3   Convenio:  236324 non-null  object 
 4   Resultado  236324 non-null  object 
dtypes: float64(2), object(3)
memory usage: 11.8+ MB


In [24]:
detalles.nunique()

Planilla     181075
Dominio:      73263
Modelo           66
Convenio:        31
Resultado         3
dtype: int64

anomalias

In [23]:
anomalias=pd.read_csv('0.0.Anomalias.COMPLETE',index_col=0)
anomalias.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 0 to 2
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Planilla       6 non-null      int64  
 1   Código         6 non-null      int64  
 2   Gravedad       6 non-null      object 
 3   Descripción    6 non-null      object 
 4   Observaciones  0 non-null      float64
dtypes: float64(1), int64(2), object(2)
memory usage: 288.0+ bytes


In [45]:
anomalias.nunique()

Planilla         199808
Código              769
Gravedad            151
Descripción         737
Observaciones     46455
dtype: int64